In [ ]:
%%capture
!pip install transformers[torch]
!pip install datasets transformers sentencepiece

In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
test = pd.read_csv("/kaggle/input/data-ready-for-models/test_ready_for_models.csv", usecols=['text_mod', 'is_sarcastic'])
train = pd.read_csv("/kaggle/input/data-ready-for-models/train_ready_for_models.csv", usecols=['text_mod', 'is_sarcastic'])

In [ ]:
train

In [ ]:
test

In [ ]:
test.rename(columns={'is_sarcastic':'label'}, inplace=True)
train.rename(columns={'is_sarcastic':'label'}, inplace=True)

In [ ]:
test = Dataset.from_pandas(test)
test = test.class_encode_column("label")
dev_test_dataset = test.train_test_split(test_size=0.5, stratify_by_column="label")

In [ ]:
train = Dataset.from_pandas(train)
train = train.class_encode_column("label")

In [ ]:
train_dev_test = DatasetDict({
    "train": train,
    "dev":dev_test_dataset["train"],
    "test": dev_test_dataset["test"]
})

In [ ]:
train_dev_test

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_ckpt = "youscan/ukr-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
tokenizer.vocab_size

In [ ]:
tokenizer.special_tokens_map

In [ ]:
def tokenize_texts(examples):
    return tokenizer(examples["text_mod"], padding=True, truncation=True, max_length=514)

In [ ]:
tokenized_train_dev_test = train_dev_test.map(tokenize_texts, batched=True)
tokenized_train_dev_test

In [ ]:
id2label = {1: "sarcastic",0: "not_sarcastic"}
label2id = {"sarcastic": 1, "not_sarcastic": 0}

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=2, label2id=label2id, id2label=id2label).to(device)

In [ ]:
predicted_labels = []
ground_truth_labels = []

# Iterate through the test data and make predictions on unfine-tuned model
for example in tokenized_train_dev_test['test']:
    text_input = example["text_mod"]

    tokenized_input = tokenizer(text_input, return_tensors="pt")

    tokenized_input = {key: value.to("cuda:0") for key, value in tokenized_input.items()}

    predictions = model(**tokenized_input)

    predicted_label = predictions.logits.argmax().item()

    predicted_labels.append(predicted_label)

    if "label" in example:
        ground_truth_labels.append(example["label"])

In [ ]:
%%capture
!pip install evaluate

In [ ]:
from evaluate import load

metrics = {
    "accuracy": load("accuracy"),
    "precision": load("precision"),
    "recall": load("recall"),
    "f1": load("f1"),
}


metric_results = {}

for metric_name, metric in metrics.items():
    result = metric.compute(predictions=predicted_labels, references=ground_truth_labels)
    metric_results[metric_name.capitalize()] = result

for metric_name, result in metric_results.items():
    print(f"{metric_name}: {result}")

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions)
    recall = recall_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    roc_auc = roc_auc_score(labels, predictions)  # Only applicable if you have probabilities

    return {"accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "roc_auc": roc_auc}

In [ ]:
from transformers import TrainingArguments

batch_size = 16
num_train_epochs = 2
logging_steps = len(tokenized_train_dev_test["train"]) // (batch_size * num_train_epochs)

args = TrainingArguments(
    output_dir="Snizhanna/ukr-roberta-base-finetuned-sarc",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.02,
    logging_steps=logging_steps,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_dev_test["train"],
    eval_dataset=tokenized_train_dev_test["dev"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
trainer.evaluate(tokenized_train_dev_test['test'])